In [ ]:
! pip install progress

In [1]:
import sys
sys.path.append('../')
import argparse, os, shutil, time, random, math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import torch.nn.functional as F
import losses
from datasets.cifar100 import get_cifar100
from train.train import get_train_fn, get_update_score_fn
from train.validate import get_valid_fn
from models.net import get_model
from losses.loss import get_loss, get_optimizer
from utils.config import parse_args, reproducibility, dataset_argument
from utils.common import make_imb_data, save_checkpoint, adjust_learning_rate


In [2]:
args = parse_args(run_type = 'jupyter')
reproducibility(args.seed)
args = dataset_argument(args)

In [3]:
args.imb_ratio = 100
args.loss_fn = 'ride'

In [4]:
try:
    assert args.num_max <= 50000. / args.num_class
except AssertionError:
    args.num_max = int(50000 / args.num_class)

N_SAMPLES_PER_CLASS = make_imb_data(args.num_max, args.num_class, args.imb_ratio)

if args.dataset == 'cifar100':
    print(f'==> Preparing imbalanced CIFAR-100')
    trainset, allset, devset, testset = get_cifar100(os.path.join(args.data_dir, 'cifar100/'), N_SAMPLES_PER_CLASS, cutout = args.cutout,  num_test = args.num_test, seed = args.seed)
    
trainloader = data.DataLoader(trainset, batch_size=args.batch_size, shuffle=True, num_workers=args.workers, drop_last=False)

allloader = data.DataLoader(allset, batch_size=args.batch_size, shuffle=False, num_workers=args.workers)
devloader = data.DataLoader(devset, batch_size=args.batch_size, shuffle=False, num_workers=args.workers)
testloader = data.DataLoader(testset, batch_size=args.batch_size, shuffle=False, num_workers=args.workers)


[500, 477, 455, 434, 415, 396, 378, 361, 344, 328, 314, 299, 286, 273, 260, 248, 237, 226, 216, 206, 197, 188, 179, 171, 163, 156, 149, 142, 135, 129, 123, 118, 112, 107, 102, 98, 93, 89, 85, 81, 77, 74, 70, 67, 64, 61, 58, 56, 53, 51, 48, 46, 44, 42, 40, 38, 36, 35, 33, 32, 30, 29, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 15, 14, 13, 13, 12, 12, 11, 11, 10, 10, 9, 9, 8, 8, 7, 7, 7, 6, 6, 6, 6, 5, 5, 5, 5]
==> Preparing imbalanced CIFAR-100
Files already downloaded and verified
Files already downloaded and verified
Magnitude set = tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], dtype=torch.int32)
Operation set = tensor([ 0,  1,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9,
         9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15], dtype=torch.int32)
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
#T

### Fix augment check

In [6]:
# Model
print ("==> creating {}".format(args.network))
model = get_model(args, N_SAMPLES_PER_CLASS)
train_criterion = get_loss(args, N_SAMPLES_PER_CLASS)
criterion = nn.CrossEntropyLoss(reduction='none') # For test, validation 
optimizer = get_optimizer(args, model)
train = get_train_fn(args)
update_score = get_update_score_fn(args)
validate = get_valid_fn(args)


############ Initial Training
test_accs, dev_accs, all_accs = [],[],[]
# for epoch in range(args.epochs):
for epoch in range(args.epochs):
    lr = adjust_learning_rate(optimizer, epoch, args)
    curr_state, label = update_score(trainloader, model, num_test = args.num_test)
    train_loss = train(args, trainloader, model, optimizer, train_criterion, epoch)


    all_loss, all_acc, all_cls = validate(args, allloader, model, criterion, N_SAMPLES_PER_CLASS,  num_class=args.num_class, mode='All Valid')
    dev_loss, dev_acc, dev_cls = validate(args, devloader, model, criterion, N_SAMPLES_PER_CLASS,  num_class=args.num_class, mode='Dev Valid')
    test_loss, test_acc, test_cls = validate(args, testloader, model, criterion, N_SAMPLES_PER_CLASS,  num_class=args.num_class, mode='Test Valid')

    print(f'Epoch: [{epoch+1} | {args.epochs}]')
    print(f'LR: {lr}')
    print(f'[Train]\tLoss:\t{train_loss:.4f}')
    print(f'[Dev set]\tLoss:\t{dev_loss:.4f}\tAcc:\t{dev_acc:.4f}')
    print(f'[Test set]\tLoss:\t{test_loss:.4f}\tAcc:\t{test_acc:.4f}')
    print(f'[All set]\tLoss:\t{all_loss:.4f}\tAcc:\t{all_acc:.4f}')
    print(f'[Dev Cls]\tMany:\t{dev_cls[0]:.4f}\tMedium:\t{dev_cls[1]:.4f}\tFew:\t{dev_cls[2]:.4f}')
    print(f'[Test Cls]\tMany:\t{test_cls[0]:.4f}\tMedium:\t{test_cls[1]:.4f}\tFew:\t{test_cls[2]:.4f}')
    print(f'[All Cls]\tMany:\t{all_cls[0]:.4f}\tMedium:\t{all_cls[1]:.4f}\tFew:\t{all_cls[2]:.4f}')


# Print the final results
print(f'Final Performance...')
print(f'best bAcc (test): {np.max(test_accs)}')
print(f'best bAcc (dev): {np.max(dev_accs)}')
print(f'best bAcc (all): {np.max(all_accs)}')


==> creating resnet32
    Total params: 0.79M
Max state: 1 // Min state: 0


ValueError: only one element tensors can be converted to Python scalars

In [ ]:
# Model
print ("==> creating {}".format(args.network))
model = get_model(args, N_SAMPLES_PER_CLASS)
train_criterion = get_loss(args, N_SAMPLES_PER_CLASS)
criterion = nn.CrossEntropyLoss()
optimizer = get_optimizer(args, model)
train = get_train_fn(args)
validate = get_valid_fn(args)

test_accs, dev_accs, all_accs = [],[],[]
# for epoch in range(args.epochs):
for epoch in range(args.epochs):
    lr = adjust_learning_rate(optimizer, epoch, args)
    train_loss = train(args, trainloader, model, optimizer, train_criterion, epoch)

    all_loss, all_acc, all_cls = validate(args, allloader, model, criterion, N_SAMPLES_PER_CLASS,  num_class=args.num_class, mode='All Valid')
    dev_loss, dev_acc, dev_cls = validate(args, devloader, model, criterion, N_SAMPLES_PER_CLASS,  num_class=args.num_class, mode='Dev Valid')
    test_loss, test_acc, test_cls = validate(args, testloader, model, criterion, N_SAMPLES_PER_CLASS,  num_class=args.num_class, mode='Test Valid')

    print(f'Epoch: [{epoch+1} | {args.epochs}]')
    print(f'LR: {lr}')
    print(f'[Train]\tLoss:\t{train_loss:.4f}')
    print(f'[Dev set]\tLoss:\t{dev_loss:.4f}\tAcc:\t{dev_acc:.4f}')
    print(f'[Test set]\tLoss:\t{test_loss:.4f}\tAcc:\t{test_acc:.4f}')
    print(f'[All set]\tLoss:\t{all_loss:.4f}\tAcc:\t{all_acc:.4f}')
    print(f'[Dev Cls]\tMany:\t{dev_cls[0]:.4f}\tMedium:\t{dev_cls[1]:.4f}\tFew:\t{dev_cls[2]:.4f}')
    print(f'[Test Cls]\tMany:\t{test_cls[0]:.4f}\tMedium:\t{test_cls[1]:.4f}\tFew:\t{test_cls[2]:.4f}')
    print(f'[All Cls]\tMany:\t{all_cls[0]:.4f}\tMedium:\t{all_cls[1]:.4f}\tFew:\t{all_cls[2]:.4f}')

    test_accs.append(test_acc)
    dev_accs.append(dev_acc)
    all_accs.append(all_acc)

# Print the final results
print(f'Final Performance...')
print(f'best bAcc (test): {np.max(test_accs)}')
print(f'best bAcc (dev): {np.max(dev_accs)}')
print(f'best bAcc (all): {np.max(all_accs)}')
